In [7]:
import urllib.request
import xml.dom.minidom

pretty_xml_as_string = ''

def fetch_xml():
    
    my_key = 'GpwTGNEp1IjyCv0Ob%2BgB4Vqhszfqr9FMFe9XXpxPTfv66tipBTYR1e1cjXAaG3su8d%2F8h5zlPkqoJY3WVra7vg%3D%3D'

    # request url정의
    url = "http://openapi.animal.go.kr/openapi/service/rest/animalShelterSrvc/shelterInfo?care_nm=&ServiceKey=" + my_key + "&pageNo=1&numOfRows=305"
    request = urllib.request.Request(url)

    # request보내기
    response = urllib.request.urlopen(request)

    # 결과 코드 정의
    rescode = response.getcode()

    if(rescode==200):
        response_body = response.read()
        dom = xml.dom.minidom.parseString(response_body.decode('utf-8'))
        
        global pretty_xml_as_string
        
        pretty_xml_as_string = dom.toprettyxml()
        print(pretty_xml_as_string)
    else:
        print("Error Code:" + rescode)

fetch_xml()

In [8]:
from bs4 import BeautifulSoup
import pandas

def make_item_list():
    soup = BeautifulSoup(pretty_xml_as_string)
    
    global ItemList
    ItemList = soup.findAll('item')
    
make_item_list()

In [9]:
def list_append():
    
    global careAddrList
    global careNmList
    global careTelList
    global orgNmList
    global saveTrgtAnimalList

    careAddrList = [] #주소
    careNmList = [] #기관명
    careTelList = [] #번호
    orgNmList = [] #지역명
    saveTrgtAnimalList = [] #취급 동물

    for item in ItemList:
        try :
            careAddr = item.find('careaddr').text
            careNm = item.find('carenm').text
            careTel = item.find('caretel').text
            orgNm = item.find('orgnm').text
            saveTrgtAnimal = item.find('savetrgtanimal').text

            careAddrList.append(careAddr)
            careNmList.append(careNm)
            careTelList.append(careTel)
            orgNmList.append(orgNm)
            saveTrgtAnimalList.append(saveTrgtAnimal)

        except AttributeError as e:
            careAddrList.append(careAddr)
            careNmList.append(careNm)
            careTelList.append(careTel)
            orgNmList.append(orgNm)
            saveTrgtAnimalList.append('기타')

list_append()

In [10]:
def make_facility_data_api():

    facility_data_api = pandas.concat([ pandas.DataFrame(careAddrList), pandas.DataFrame(careNmList), pandas.DataFrame(careTelList), 
                                   pandas.DataFrame(orgNmList), pandas.DataFrame(saveTrgtAnimalList) ], axis=1)
    
    facility_data_api.to_csv("./facility_data_api.csv", encoding='utf-8-sig')
    
make_facility_data_api()

In [11]:
import pymysql
import datetime

In [12]:
import pickle
import csv

def setup_db():

    MYSQL_USER_DATA_SAVED_FILE = "res/mysql/userinfo"
    
    global f
    f = open(MYSQL_USER_DATA_SAVED_FILE, 'rb')
    mysql_user_info = pickle.load(f)
    f.close()
    
    global db
    db = pymysql.connect(
        # localhost와 같은 아이피 주소임
        host = '127.0.0.1',
        # MySQL의 포트번호
        port = 3306,
        # 아이디
        user = mysql_user_info['user_id'],
        # 비밀번호
        passwd = mysql_user_info['password'],
        # 사용하려는 스키마
        db = 'petto_db',
        # 캐릭터 셋 설정
        charset = 'utf8'
    )
    
    global cursor
    cursor = db.cursor()
    
    f = open('facility_data_api.csv', 'r', encoding='UTF8')
    
    global csv_reader_facility
    csv_reader_facility = csv.reader(f)

    print(csv_reader_facility)
    
setup_db()

In [13]:
def save_in_db():
    ######################## 매일 자정마다 리스트 초기화
    
    initialization_sql = """truncate table petto_facility""" 
    
    cursor.execute(initialization_sql)
    
    db.commit()
    
    ########################
    
    cnt = 0

    for row in csv_reader_facility:
        if cnt == 0:
            cnt += 1
            continue

        facility_addr = (row[1])
        facility_name = (row[2])
        facility_tel = (row[3])
        org_name = (row[4])
        save_trgt_animal = (row[5])


        sql = """
                insert into petto_facility (facility_addr, facility_name, facility_tel, org_name, save_trgt_animal) values 
                (%s, %s, %s, %s, %s)
            """

        cursor.execute(sql, (facility_addr, facility_name, facility_tel, org_name, save_trgt_animal))

    db.commit()

    f.close()
    db.close()
    print('db 저장 완료')
    
save_in_db()

db 저장 완료


In [6]:
!pip install schedule

In [1]:
import schedule
import time
import datetime

In [2]:
def fetch_facility_list():
    
    fetch_xml()
    make_item_list()
    list_append()
    make_facility_data_api()
    setup_db()
    save_in_db()
    
schedule.every().day.at("00:00").do(fetch_facility_list)
 
#실제 실행하게 하는 코드
while True:
    schedule.run_pending()
    time.sleep(1)


KeyboardInterrupt: 